In [1]:
%pip install crewai_tools langchain_community langchain_google_genai fpdf2 markdown2

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install crewai agentic-ai

Note: you may need to restart the kernel to use updated packages.


In [3]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

ModuleNotFoundError: No module named 'google.generativeai'

In [4]:
# Warning Control
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Import crewAI library
from crewai import Agent, Task, Crew, Process
from crewai.project import CrewBase, agent, crew, task
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List
from crewai.flow.flow import Flow, listen, start

In [6]:
import os

# %pip install python-dotenv

from dotenv import load_dotenv
load_dotenv()

apiKey = os.getenv("GOOGLE_API_KEY")

# print(apiKey)

In [7]:
#setting up LLM

from crewai import LLM

llm = LLM(model="gemini/gemini-2.5-pro",
                             verbose=True,
                             temperature=0.5,
                             api_key = apiKey)

# print(apiKey)


In [18]:
%pip install --upgrade "crewai[tools]"

  Using cached crewai_tools-0.58.0-py3-none-any.whl.metadata (10 kB)
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
INFO: pip is looking at multiple versions of crewai-tools to determine which version is compatible with other requirements. This could take a while.
  Using cached portalocker-2.7.0-py2.py3-none-any.whl.metadata (6.8 kB)
  Using cached onnxruntime-1.22.0-cp313-cp313-win_amd64.whl.metadata (5.0 kB)
  Using cached litellm-1.74.3-py3-none-any.whl.metadata (40 kB)
  Using cached json_repair-0.25.2-py3-none-any.whl.metadata (7.9 kB)
  Using cached crewai-0.150.0-py3-none-any.whl.metadata (35 kB)
  Using cached crewai-0.148.0-py3-none-any.whl.metadata (35 kB)
  Using cached litellm-1.72.6-py3-none-any.whl.metadata (39 kB)
  Using cached crewai_tools-0.55.0-py3-none-any.whl.metadata (10 kB)
  Using cached crewai-0.141.0-py3-none-any.whl.metadata (35 kB)
  Using cached crewai_tools-0.51.1-py3-none-a

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      + c:\Users\akikumar\AppData\Local\Programs\Python\Python313\python.exe C:\Users\akikumar\AppData\Local\Temp\1\pip-install-2e8ofil_\numpy_03aefcef60db4cfd874df0df87a967f6\vendored-meson\meson\meson.py setup C:\Users\akikumar\AppData\Local\Temp\1\pip-install-2e8ofil_\numpy_03aefcef60db4cfd874df0df87a967f6 C:\Users\akikumar\AppData\Local\Temp\1\pip-install-2e8ofil_\numpy_03aefcef60db4cfd874df0df87a967f6\.mesonpy-0uzqk0z0 -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\akikumar\AppData\Local\Temp\1\pip-install-2e8ofil_\numpy_03aefcef60db4cfd874df0df87a967f6\.mesonpy-0uzqk0z0\meson-python-native-file.ini
      The Meson build system
      Version: 1.2.99
      Source dir: C:\Users\akikumar\AppData\Local\Temp\1\pip-install-2e8ofil_\numpy_03aefcef60db4cfd874df0df87a967f6
      Build dir: C:\Users\akikumar\App

In [22]:
import os
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Hide the root tkinter window
Tk().withdraw()

# Open file dialog
file_path = askopenfilename(
    title="Select ABAP Code File",
    filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")]
)

abap_code_example = ""

if file_path:
    with open(file_path, 'r') as f:
        abap_code_example = f.read()
    print("File selected:", file_path)
    print("Code Preview:\n", abap_code_example[:500])  # Show a preview
else:
    print("No file selected.")

File selected: C:/Users/akikumar/Downloads/OneDrive_2025-07-28/FSTS Gemini/FS and TS/RTR/Code/1.ZGRTR_FI_HRC_CREDIT_UPD.txt
Code Preview:
 *&---------------------------------------------------------------------*
*& Report ZGRTR_FI_HRC_CREDIT_UPD
*&---------------------------------------------------------------------*
*&
*&---------------------------------------------------------------------*
REPORT zgrtr_fi_hrc_credit_upd.

*$-Top Include
INCLUDE zgrtr_fi_hrc_credit_top.

*$-Selection-screen
INCLUDE zgrtr_fi_hrc_credit_sel.

*$- Routines
INCLUDE zgrtr_fi_hrc_credit_f01.

* Begin of chg by Mohit on 26 Sep 2024
*$====================


In [23]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI
# from crewai_tools import WebsiteSearchTool




# --- Agent Definitions ---

# 1. ABAP Code Interpreter (No changes needed)
abap_interpreter = Agent(
    role='ABAP Code Interpreter',
    goal='Analyze the provided ABAP code to understand its structure, main components (like FORMs, METHODs, FUNCTIONs), and the overall program flow.',
    backstory=(
        "As a seasoned ABAP developer with decades of experience, you have an unparalleled ability to read and instantly "
        "comprehend even the most complex and archaic ABAP code. You can see beyond the syntax to the underlying business "
        "logic and program structure. Your task is to distill this complex code into a clear, structured summary that "
        "other agents can use."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 2. Technical Logic Reviewer (Slightly re-focused)
# This agent now focuses STRICTLY on the technical implementation flow.
technical_logic_reviewer = Agent(
    role='Technical Logic Reviewer',
    goal='Review the structured summary of the ABAP code to map out the detailed technical logic, including specific subroutine calls, conditional statements, and loops.',
    backstory=(
        "You are a meticulous systems analyst who reverse-engineers legacy code. Your strength is in tracing the exact "
        "execution path, identifying every subroutine call (PERFORM, CALL FUNCTION), every conditional branch (IF/ELSE/CASE), "
        "and every loop (DO, WHILE) to create a precise, code-level flow map."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 3. NEW AGENT: Business Process Summarizer
# This agent's purpose is to create the high-level abstraction.
business_process_summarizer = Agent(
    role='Business Process Summarizer',
    goal='Analyze the technical summary of the ABAP code to interpret and describe the high-level business process. Translate technical jargon into business activities.',
    backstory=(
        "You are a business analyst with a talent for bridging the gap between technical teams and business stakeholders. "
        "You can look at a technical workflow and explain what it *does* in simple business terms. For example, you translate 'SELECT from VBAK, VBAP' into 'Retrieve Sales Order Details' "
        "and 'PERFORM apply_discount' into 'Calculate Customer Discount'."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# mermaid_docs_tool = WebsiteSearchTool(website_url='https://mermaid.js.org/syntax/flowchart.html')

# 4. Diagram Blocks Generator (No changes needed)
diagram_blocks_generator = Agent(
    role='Workflow Diagram Architect',
    goal='Translate a description of a process into a list of abstract diagram components (e.g., Start, End, Process, Decision, IO).',
    backstory=(
        "You are a systems architect who thinks visually. You can take a description of a process and immediately see it as a "
        "flowchart. Your focus is on defining the fundamental building blocks—the nodes and their types—before they are rendered."
    ),
    verbose=True,
    # tools=[mermaid_docs_tool],
    allow_delegation=False,
    llm=llm
)

# 5. Diagram Generation Agent (No changes needed)
# mermaid_docs_tool = ScrapeWebsiteTool(website_url='https://mermaid.js.org/syntax/flowchart.html')

diagram_generator = Agent(
    role='Mermaid Code Generator',
    goal='Generate syntactically correct MermaidJS flowchart code from a list of abstract diagram components.',
    backstory=(
        "You are an expert in creating diagrams-as-code using Mermaid. You have a deep understanding of Mermaid's syntax "
        "for generating flowcharts. You can translate structured data into clean, readable, and accurate Mermaid code."
    ),
    # tools=[mermaid_docs_tool],
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 6. Manager Agent (The Final Quality Gate)
# Now with clear delegation capability to enforce quality.
manager = Agent(
    role='Quality Assurance Manager',
    goal='Review the final MermaidJS diagram to ensure it accurately represents the original ABAP code\'s logic at the requested level of detail (either technical or high-level). Provide the final, approved diagram code.',
    backstory=(
        "With a dual background in software development and project management, you are the ultimate quality gatekeeper. "
        "You ensure that the final product meets the requirements. You look at the generated diagram and the initial analysis and ask: 'Does this flowchart accurately reflect the workflow?' "
        "You have the final say and can request revisions if the diagram is incorrect, confusing, or does not meet the required level of abstraction."
    ),
    verbose=True,
    allow_delegation=True, # IMPORTANT: This allows the manager to send work back to other agents.
    llm=llm
)

In [25]:
# === Technical Workflow Tasks ===
technical_task_1 = Task(
    description=f"Analyze this ABAP code and create a structural summary:\n\n{abap_code_example}",
    expected_output="A summary of all FORMs, METHODs, and the main program flow.",
    agent=abap_interpreter
)

technical_task_2 = Task(
    description="Based on the code summary, map out the detailed technical logic. Trace every PERFORM, IF/ELSE branch, and LOOP.",
    expected_output="A step-by-step technical workflow, mentioning specific FORM names and conditions.",
    agent=technical_logic_reviewer,
    context=[technical_task_1]
)

technical_task_3 = Task(
    description="Convert the detailed technical workflow into abstract diagram blocks (Process, Decision, IO, etc.).",
    expected_output="A list of diagram components representing the technical flow.",
    agent=diagram_blocks_generator,
    context=[technical_task_2]
)

technical_task_4 = Task(
    description="Generate the MermaidJS flowchart code from the abstract blocks. Use technical names from the code in the diagram nodes.",
    expected_output="A complete, syntactically correct MermaidJS flowchart code block.",
    agent=diagram_generator,
    context=[technical_task_3]
)

technical_task_5 = Task(
    description="Review the generated technical MermaidJS code. Ensure it accurately reflects the detailed code flow, including all FORMs and key decision points. Provide the final, approved MermaidJS code as your answer.",
    expected_output="The final, validated MermaidJS code for the technical flowchart.",
    agent=manager,
    context=[technical_task_4]
)

technical_tasks = [
    technical_task_1,
    technical_task_2,
    technical_task_3,
    technical_task_4,
    technical_task_5
]

# === High-Level Business Flowchart Tasks ===
high_task_1 = Task(
    description=f"Analyze this ABAP code and create a structural summary:\n\n{abap_code_example}",
    expected_output="A summary of all FORMs, METHODs, and the main program flow.",
    agent=abap_interpreter
)

high_task_2 = Task(
    description="Based on the code summary, interpret the business process. Translate technical actions into plain business terms.",
    expected_output="A high-level summary of the business process in simple language, ignoring technical details.",
    agent=business_process_summarizer,
    context=[high_task_1]
)

high_task_3 = Task(
    description="Convert the high-level business process description into abstract diagram blocks (Process, Decision, etc.).",
    expected_output="A list of diagram components representing the business process.",
    agent=diagram_blocks_generator,
    context=[high_task_2]
)

high_task_4 = Task(
    description="Generate the MermaidJS flowchart code from the abstract business blocks. Use clear, simple business language for the diagram nodes.",
    expected_output="A complete, syntactically correct MermaidJS flowchart code block.",
    agent=diagram_generator,
    context=[high_task_3]
)

high_task_5 = Task(
    description="Review the generated high-level MermaidJS code. Ensure it represents the business purpose of the code and is easy for a non-technical person to understand. Provide the final, approved MermaidJS code as your answer.",
    expected_output="The final, validated MermaidJS code for the high-level business flowchart.",
    agent=manager,
    context=[high_task_4]
)

high_level_tasks = [
    high_task_1,
    high_task_2,
    high_task_3,
    high_task_4,
    high_task_5
]

# --- Crew Execution (Unchanged) ---

# Create and run the Technical Crew
technical_crew = Crew(
    agents=[abap_interpreter, technical_logic_reviewer, diagram_blocks_generator, diagram_generator, manager],
    tasks=technical_tasks,
    process=Process.sequential,
    verbose=True # Changed from 2 to True
)

# Create and run the High-Level Crew
high_level_crew = Crew(
    agents=[abap_interpreter, business_process_summarizer, diagram_blocks_generator, diagram_generator, manager],
    tasks=high_level_tasks,
    process=Process.sequential,
    verbose=True # Changed from 2 to True
)

# Kick off the technical flowchart generation
print("🚀 Starting Technical Flowchart Generation...")
technical_result = technical_crew.kickoff()
print("\n\n✅ Technical Flowchart Generation Complete!")
print("Final Approved Technical Mermaid Code:")
print(technical_result)


# Kick off the high-level flowchart generation
print("\n\n🚀 Starting High-Level Business Flowchart Generation...")
high_level_result = high_level_crew.kickoff()
print("\n\n✅ High-Level Business Flowchart Generation Complete!")
print("Final Approved High-Level Mermaid Code:")
print(high_level_result)

🚀 Starting Technical Flowchart Generation...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 48ed77bf-9e05-40ce-bc03-b640ae1489e1                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: ABAP Code Interpreter                                                                                   │
│                                                                                                                 │
│  Task: Analyze this ABAP code and create a structural summary:                                                  │
│                                                                                                                 │
│  *&---------------------------------------------------------------------*                                       │
│  *& Report ZGRTR_FI_HRC_CREDIT_UPD                                                                              │
│  *&---------------------------------------------------------------------*                                       │
│  *&                                                                                                             │
│  *&---------------------------------------------------------------------*                                       │
│  REPORT zgrtr_fi_hrc_credit_upd.                                                                                │
│                                                                                                                 │
│  *$-Top Include                                                                                                 │
│  INCLUDE zgrtr_fi_hrc_credit_top.                                                                               │
│                                                                                                                 │
│  *$-Selection-screen                                                                                            │
│  INCLUDE zgrtr_fi_hrc_credit_sel.                                                                               │
│                                                                                                                 │
│  *$- Routines                                                                                                   │
│  INCLUDE zgrtr_fi_hrc_credit_f01.                                                                               │
│                                                                                                                 │
│  * Begin of chg by Mohit on 26 Sep 2024                                                                         │
│  *$=======================================================$*                                                    │
│  *$ At-Selection-Screen On Value Request                   *                                                    │
│  *$=======================================================$*                                                    │
│  *-F4 help for input file                                                                                       │
│  AT SELECTION-SCREEN ON VALUE-REQUEST FOR p_ppath.                                                              │
│    IF r_ps IS NOT INITIAL.                                                                                      │
│      PERFORM f4help_pres_server USING p_ppath.  "Presentation File path                                         │
│    ENDIF.                                                                                                       │
│                                                                                                                 │
│  * End of chg                                                                                                   │
│                                                                                                                 │
│  *$====================================================

HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by 
SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get 
local issuer certificate (_ssl.c:1028)')))

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: ABAP Code Interpreter                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### **Program Analysis: ZGRTR_FI_HRC_CREDIT_UPD**                                                              │
│                                                                                                                 │
│  #### **1. Program Description**                                                                                │
│                                                                                                                 │
│  The ABAP report `ZGRTR_FI_HRC_CREDIT_UPD` is a "Credit Data Update Engine". Its primary function is to read    │
│  credit-related data from a file and update the corresponding information within the SAP system. The program    │
│  is designed to handle files from both the presentation server (user's local machine) and the application       │
│  server. It includes robust input validation, file processing logic, and detailed logging of its activities     │
│  using the standard Application Log framework.                                                                  │
│                                                                                                                 │
│  #### **2. Program Structure**                                                                                  │
│                                                                                                                 │
│  The program is modularized using `INCLUDE` statements, which separate data declarations, the selection         │
│  screen, and the processing logic (subroutines).                                                                │
│                                                                                                                 │
│  *   **`zgrtr_fi_hrc_credit_top`**: This include contains all global data declarations. This typically          │
│  includes `TABLES`, `TYPES` structures, and `DATA` variable definitions used throughout the program.            │
│  *   **`zgrtr_fi_hrc_credit_sel`**: This include defines the user interface, the `SELECTION-SCREEN`. It         │
│  contains the `PARAMETERS` and `SELECT-OPTIONS` that allow the user to specify input criteria, such as file     │
│  paths and processing options.                                                                                  │
│  *   **`zgrtr_fi_hrc_credit_f01`**: This include contains the implementation of all the subroutines             │
│  (`FORM`...`ENDFORM` blocks) that are called from the main program logic. This is where the core business       │
│  logic resides.                                                                                                 │
│                                                                                                                 │
│  #### **3. Main Program Flow & Events**                                                                         │
│                                                                                                                 │
│  The program execution follows the standard ABAP report event model.                                            │
│                                                                                                                 │
│  1.  **`INITIALIZATION`**:                                                                                      │
│      *   This event fires before the selection screen i

...

HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by 
SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get 
local issuer certificate (_ssl.c:1028)')))

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.InternalServerError: litellm.InternalServerError: VertexAIException - {                         │
│    "error": {                                                                                                   │
│      "code": 503,                                                                                               │
│      "message": "The model is overloaded. Please try again later.",                                             │
│      "status": "UNAVAILABLE"                                                                                    │
│    }                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 914a3b81-7cc2-4e8b-bbc7-eab9b35f440b                                                                     │
│  Agent: Quality Assurance Manager                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 48ed77bf-9e05-40ce-bc03-b640ae1489e1                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

InternalServerError: litellm.InternalServerError: litellm.InternalServerError: VertexAIException - {
  "error": {
    "code": 503,
    "message": "The model is overloaded. Please try again later.",
    "status": "UNAVAILABLE"
  }
}


In [ ]:
# Wrap the mermaid code in a minimal HTML with CDN
html_content = f"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Mermaid Diagram</title>
  <script type="module">
    import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs';
    mermaid.initialize({{ startOnLoad: true }});
  </script>
</head>
<body>
  <div class="mermaid">
    {technical_result}
    {high_level_result}
  </div>
</body>
</html>
"""

# Save the HTML to a file in the current directory
file_path = 'mermaid_diagram1.html'
with open(file_path, 'w') as f:
    f.write(html_content)

print(f"HTML file saved to {file_path}. Open it in your browser to view the diagram.")
